# OLDP Client Demo

This is a beginners level demo that shows how to use the OLDP SDK for Python, see https://github.com/openlegaldata/oldp-sdk-python.

You can install the SDK by running:
```
pipenv install git+https://github.com/openlegaldata/oldp-sdk-python.git#egg=oldp-api
```

Let's begin with creating an instance of the API Client.

In [ ]:
import oldp_client 

conf = oldp_client.Configuration()
conf.api_key['api_key'] = '123abc'  # Replace this with your API key
api_client = oldp_client.ApiClient(conf)

The API endpoints are documented [here](https://github.com/openlegaldata/oldp-sdk-python#documentation-for-api-endpoints).

We follow this guide to get an instance of the API for courts.

In [ ]:
courts_api = oldp_client.CourtsApi(api_client)

Next we show how to query the data OLDP has about courts.

In [ ]:
courts = courts_api.courts_list().results
print("The API call returned {} courts.".format(len(courts)))
print("Example court:\n{}".format(courts[10]))

The value for city and state is an id of a separate table itself, that can be looked up in the respective APIs.

In [ ]:
cities_api = oldp_client.CitiesApi(api_client)
states_api = oldp_client.StatesApi(api_client)

print("City: {}".format(cities_api.cities_read(courts[10].city)))
print("State: {}".format(states_api.states_read(courts[10].state)))

Finally we learn how to do some visualizations. For that we first get all cases of one state. Then we plot the distribution of the cases on the different courts, as well as a wordcloud.

In [ ]:
cases_api = oldp_client.CasesApi(api_client)

def get_cases_for_state(state_id):
    cases_per_court = {}
    cases = []
    
    courts = courts_api.courts_list(state_id=state_id).results
    for court in courts:
        cases_list = cases_api.cases_list(court_id=court.id).results
        cases_per_court[court.id] = cases_list
        cases += cases_list
        
    return cases, cases_per_court

In [ ]:
import matplotlib.pylab as plt

_, cases_per_court = get_cases_for_state(3)

for court_id, cases in cases_per_court:
    cases_per_court[court_id] = len(cases)

# sort according to the number of cases
sorted(cases_per_court.items(), operator.itemgetter(1))

x, y = zip(*)
plt.plot(x, y)
plt.show()

In [ ]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from wordcloud import WordCloud

text = ''
cases, _ = get_cases_for_state(3)
for case in cases:
    # get clean text from the HTML document
    soup = BeautifulSoup(case.content)
    text += soup.get_text().strip()

wordcloud = WordCloud(max_font_size=60).generate(text)
plt.figure(figsize=(16,12))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()